In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../../')
from noise2seg.models import Noise2Seg, Noise2SegConfig
import numpy as np
from csbdeep.utils import plot_history
from noise2seg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from noise2seg.utils.seg_utils import *
from noise2seg.utils.compute_precision_threshold import measure_precision
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy import ndimage
import tensorflow as tf
import keras.backend as K
import urllib
import os
import zipfile
import pandas as pd

Using TensorFlow backend.


#### Read data and load model

In [2]:
def read_data(exp, noise_level):
    
    data_path = '/home/prakash/Desktop/fileserver_Noise2Seg/data/'
    gt_data = np.load(data_path+exp+'_n0'+'/test/test_data.npz', allow_pickle=True)
    test_data = np.load(data_path+exp+'_'+noise_level+'/test/test_data.npz', allow_pickle=True)
    gt_data = gt_data['X_test']
    test_data = test_data['X_test']
    return gt_data, test_data

In [3]:
def load_trained_model(exp, noise_level, run, fraction):
    
    exp_path = '/home/prakash/Desktop/fileserver_Noise2Seg/experiments/'
    base_dir = exp_path+exp+'_'+noise_level+'_run'+str(run)+'/fraction_'+fraction+'/'
    model_name = exp+'_'+noise_level+'_run'+str(run)+'_model'
    n2s_model = Noise2Seg(None, model_name, base_dir)
    return n2s_model

#### PSNR computation

In [4]:
def PSNR(gt, img):
    mse = np.mean(np.square(gt - img))
    return 20 * np.log10(np.max(gt)-np.min(gt)) - 10 * np.log10(mse)

In [5]:
def compute_mean_psnr(exp, noise_level, fraction, gt_data, test_data):
    psnr_per_run = []
    runs = np.arange(1,6)
    for run_idx in runs:
        n2s_model = load_trained_model(exp, noise_level, run_idx, fraction)
        denoised_images = []
        for i in range(test_data.shape[0]):
            denoised_ = n2s_model.predict(test_data[i].astype(np.float32),'YX')[...,0]
            denoised_images.append(denoised_)
        denoised_images = np.array(denoised_images)

        psnrs = []
        for gt, img in zip(gt_data, denoised_images):
            psnrs.append(PSNR(gt, img))

        psnrs = np.array(psnrs)
        psnr_per_run.append(np.round(np.mean(psnrs), 2))
    return np.mean(psnr_per_run)

#### Compute results

In [6]:
gt_data_n20, test_data_n20 = read_data('DSB2018', 'n20')
gt_data_n10, test_data_n10 = read_data('DSB2018', 'n10')
alpha_05_dsb_n20 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n20', fraction='0.25', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
alpha_05_dsb_n10 = compute_mean_psnr(exp='alpha0.5_dsb', noise_level='n10', fraction='0.25', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_dsb_n10 = compute_mean_psnr(exp='finDenoise_dsb', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_dsb_n20 = compute_mean_psnr(exp='finDenoise_dsb', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)

Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [7]:
gt_data_n20, test_data_n20 = read_data('Flywing', 'n20')
gt_data_n10, test_data_n10 = read_data('Flywing', 'n10')
alpha_05_flywing_n20 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n20', fraction='0.125', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
alpha_05_flywing_n10 = compute_mean_psnr(exp='alpha0.5_flywing', noise_level='n10', fraction='0.125', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_flywing_n10 = compute_mean_psnr(exp='finDenoise_flywing', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_flywing_n20 = compute_mean_psnr(exp='finDenoise_flywing', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)

Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


In [8]:
gt_data_n20, test_data_n20 = read_data('Mouse', 'n20')
gt_data_n10, test_data_n10 = read_data('Mouse', 'n10')
alpha_05_mouse_n20 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n20', fraction='0.125', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)
alpha_05_mouse_n10 = compute_mean_psnr(exp='alpha0.5_nmouse', noise_level='n10', fraction='0.125', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_mouse_n10 = compute_mean_psnr(exp='finDenoise_nmouse', noise_level='n10', fraction='100.0', 
                                        gt_data=gt_data_n10, test_data=test_data_n10)
n2v_mouse_n20 = compute_mean_psnr(exp='finDenoise_nmouse', noise_level='n20', fraction='100.0', 
                                        gt_data=gt_data_n20, test_data=test_data_n20)

Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.


#### Create table

In [24]:
styles = [dict(selector="caption", 
    props=[("text-align", "center"),
    ("font-size", "120%"),
    ("color", 'black')])] 

In [25]:
heads = ['dsb N2V', 'dsb DenoiSeg', 'flywing N2V', 'flywing DenoiSeg', 'mouse N2V', 'mouse DenoiSeg']
datasets = ['N2V', 'DenoiSeg', 'N2V', 'DenoiSeg', 'N2V', 'DenoiSeg']

noise_levels = ['n10', 'n20']
n10_scores = [n2v_dsb_n10, alpha_05_dsb_n10, n2v_flywing_n10, alpha_05_flywing_n10, n2v_mouse_n10, alpha_05_mouse_n10]
n20_scores = [n2v_dsb_n20, alpha_05_dsb_n20, n2v_flywing_n20, alpha_05_flywing_n20, n2v_mouse_n20, alpha_05_mouse_n20]
scores = np.array([n10_scores, n20_scores])
df = pd.DataFrame(scores,index=noise_levels,columns=heads)
df.style.set_caption('Mean PSNR in dB, for DenoiSeg alpha = 0.5').set_table_styles(styles)

,dsb N2V,dsb DenoiSeg,flywing N2V,flywing DenoiSeg,mouse N2V,mouse DenoiSeg
n10,38.010000,37.572000,33.164000,33.126000,37.862000,37.420000
n20,35.536000,35.384000,30.720000,30.456000,34.596000,34.216000
